In [1]:
from PyQt5 import QtCore, QtWidgets
from PyQt5.QtWidgets import QApplication, QMessageBox
from PyQt5.QtWebEngineWidgets import QWebEngineView
import requests
import os
import sys

In [2]:
class WeatherWorker(QtCore.QThread):
    weather_fetched = QtCore.pyqtSignal(str)

    def __init__(self, latitude, longitude):
        super().__init__()
        self.latitude = latitude
        self.longitude = longitude

    def run(self):
        self.weather_info = self.get_weather_data()
        self.weather_fetched.emit(self.weather_info)

    def get_weather_data(self):
        # Get grid points based on latitude and longitude
        url = f"https://api.weather.gov/points/{self.latitude},{self.longitude}"
        response = requests.get(url)
        if response.status_code != 200:
            return "Error fetching grid points."

        grid_data = response.json()
        forecast_url = grid_data['properties']['forecast']

        # Fetch weather forecast data from the NWS API
        response = requests.get(forecast_url)
        if response.status_code != 200:
            return "Could not retrieve weather data."

        forecast_data = response.json()
        periods = forecast_data['properties']['periods']
        
        # Retrieve temperature and wind speed from the first forecast period
        temp = periods[0]['temperature']
        wind_speed = periods[0]['windSpeed']
        return f"Temperature: {temp}°F, Wind: {wind_speed}"

In [3]:
class Ui_MainWindow():
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(500, 400)

        self.weather_button = QtWidgets.QPushButton(MainWindow)
        self.weather_button.setGeometry(QtCore.QRect(60, 50, 150, 30))
        self.weather_button.setObjectName("weather_button")

        self.status_label = QtWidgets.QLabel(MainWindow)
        self.status_label.setGeometry(QtCore.QRect(60, 100, 400, 30))
        self.status_label.setObjectName("status_label")

        # Ideal Weather Input
        self.ideal_temp_input = QtWidgets.QLineEdit(MainWindow)
        self.ideal_temp_input.setGeometry(QtCore.QRect(60, 150, 100, 30))
        self.ideal_temp_input.setPlaceholderText("Ideal Temp (°F)")

        self.ideal_wind_input = QtWidgets.QLineEdit(MainWindow)
        self.ideal_wind_input.setGeometry(QtCore.QRect(180, 150, 100, 30))
        self.ideal_wind_input.setPlaceholderText("Ideal Wind (mph)")
      
        self.check_weather_button = QtWidgets.QPushButton(MainWindow)
        self.check_weather_button.setGeometry(QtCore.QRect(60, 200, 150, 30))
        self.check_weather_button.setObjectName("check_weather_button")

        self.ideal_weather_label = QtWidgets.QLabel(MainWindow)  # New label for ideal weather results
        self.ideal_weather_label.setGeometry(QtCore.QRect(60, 250, 400, 60))  # Positioned below the button
        self.ideal_weather_label.setWordWrap(True)
        self.ideal_weather_label.setObjectName("ideal_weather_label")
        self.exit_button = QtWidgets.QPushButton(MainWindow)  # Added Exit Button
        self.exit_button.setGeometry(QtCore.QRect(250, 200, 150, 30))
        self.exit_button.setObjectName("exit_button")

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

        self.exit_button.clicked.connect(MainWindow.close)  # Connect Exit Button

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "Sweater Weather - Homepage"))
        self.weather_button.setText(_translate("MainWindow", "Get Weather Data"))
        self.status_label.setText(_translate("MainWindow", "Status: Ready"))
        self.check_weather_button.setText(_translate("MainWindow", "Check Ideal Weather"))
        self.exit_button.setText(_translate("MainWindow", "Exit"))  # Set Exit Button Text

In [4]:
class MainWindow(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self)

        self.ui.weather_button.clicked.connect(self.fetch_weather)
        self.ui.check_weather_button.clicked.connect(self.check_ideal_weather)

    def fetch_weather(self):
        self.weather_worker = WeatherWorker(32.96, -96.73)  # Latitude and Longitude for Richardson
        self.weather_worker.weather_fetched.connect(self.update_weather_info)
        self.weather_worker.start()  # Start the worker thread

    def update_weather_info(self, weather_info):
        self.ui.status_label.setText(weather_info)

    def check_ideal_weather(self):
        self.weather_worker = WeatherWorker(32.96, -96.73)
        self.weather_worker.weather_fetched.connect(self.compare_weather)
        self.weather_worker.start()

    def compare_weather(self, current_weather):
        try:
            # Get ideal values from input fields
            ideal_temp = float(self.ui.ideal_temp_input.text())
            ideal_wind = float(self.ui.ideal_wind_input.text())
          
        except ValueError:
            QMessageBox.warning(self, "Input Error", "Please enter valid numeric values for ideal weather.")
            return
        # Check current weather
        current_data = current_weather.split(", ")
        current_temp = float(current_data[0].split(": ")[1][:-2])  # Extract temperature
        current_wind = float(current_data[1].split(": ")[1].split(" ")[0])  # Extract wind speed

        # Check if current conditions match ideal conditions
        temp_match = ideal_temp - 5 <= current_temp <= ideal_temp + 5
        wind_match = ideal_wind - 5 <= current_wind <= ideal_wind + 5

        # Feedback on temperature comparison
        if not temp_match:
            if current_temp > ideal_temp + 5:
                temp_feedback = f"Too hot! Current Temp: {current_temp}°F."
            else:
                temp_feedback = f"Too cold! Current Temp: {current_temp}°F."
        else:
            temp_feedback = f"Temperature is within the ideal range: {current_temp}°F."

        # Feedback on wind comparison
        if not wind_match:
            if current_wind > ideal_wind + 5:
                wind_feedback = f"Wind speed too high! Current Wind: {current_wind} mph."
            else:
                wind_feedback = f"Wind speed too low! Current Wind: {current_wind} mph."
        else:
            wind_feedback = f"Wind speed is within the ideal range: {current_wind} mph."
        
        # Combine results
        result = f"{temp_feedback}\n{wind_feedback}"

        # Final message on whether conditions are ideal
        if temp_match and wind_match:
            result += "\nIdeal weather conditions met!"
        else:
            result += "\nIdeal weather conditions not met."
            # Update the label with the result
        self.ui.ideal_weather_label.setText(result)  # Display results in the new label

In [5]:
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_window = MainWindow()  # Create an instance of MainWindow (the class you defined above)
    main_window.show()
    sys.exit(app.exec_())

SystemExit: 0

C:\Users\Gayathri\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
